In [57]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from raug.checkpoints import load_model
from my_model import set_model
from PIL import Image
from aug_isic import ImgEvalTransform
import torch
import numpy as np
import cv2
import os

In [74]:
models = [
    {
        "name": "resnest101e",
        "path": "best_results/DA2_dull_razor_shades_of_gray_cropped_images_folder/resnest101e_fold-1_lrinit-0.0001_batchsize-8_optimizer-AdamW_maxepochs-150_DA-2_PPen-None_PPha-dull_razor_PPco-shades_of_gray_PPde-None_PPno-True_PPcr-cropped_images_folder_PPre-True_drop-0.0_trainjustclassifier-False_1681332137441572/best-checkpoint/best-checkpoint.pth"
    },
    {
        "name": "efficientnet_b2",
        "path": "best_results/DA2_dull_razor_shades_of_gray_cropped_images_folder/efficientnet_b2_fold-1_lrinit-0.0001_batchsize-8_optimizer-Adam_maxepochs-150_DA-2_PPen-None_PPha-dull_razor_PPco-shades_of_gray_PPde-None_PPno-True_PPcr-cropped_images_folder_PPre-True_drop-0.0_trainjustclassifier-False_16813227212758167/best-checkpoint/best-checkpoint.pth"
    },
    {
        "name": "tf_efficientnet_b5",
        "path": "best_results/DA2_dull_razor_shades_of_gray_cropped_images_folder/tf_efficientnet_b5_fold-1_lrinit-0.0001_batchsize-8_optimizer-AdamW_maxepochs-150_DA-2_PPen-None_PPha-dull_razor_PPco-shades_of_gray_PPde-None_PPno-True_PPcr-cropped_images_folder_PPre-True_drop-0.0_trainjustclassifier-False_16812917987950535/best-checkpoint/best-checkpoint.pth"
    }
]

# imagem que nenhuma rede acertou (classe 0): ISIC_0016072
# imagem que somente effb2 acertou (classe 1): ISIC_0013321
# imagem que somente effb5 acertou (classe 1): ISIC_0013842
# imagem que somente as 3 redes acertaram (classe 0): ISIC_0015447
# imagem que todas as redes acertaram (classe 2): ISIC_0012786

lesion_images = [
    {
        "img_id": "ISIC_0014434",
        "class": 0,
        "guess": "all_guessed_wrong"
    },
    {
        "img_id": "ISIC_0013321",
        "class": 1,
        "guess": "only_effb2_guessed_right"
    },
    {
        "img_id": "ISIC_0013842",
        "class": 1,
        "guess": "only_effb5_guessed_right"
    },
    {
        "img_id": "ISIC_0015447",
        "class": 0,
        "guess": "just_the_3_guessed_right"
    },
    {
        "img_id": "ISIC_0015167",
        "class": 1,
        "guess": "all_nets_guessed_right"
    }
]

for lesion_image in lesion_images:
    img_path = f"/home/a52550/Desktop/datasets/ISIC2017/test/cropped_images/{lesion_image['img_id']}.jpg"

    i = Image.open(img_path).convert("RGB")

    if not os.path.exists(f"gradcam/{lesion_image['guess']}"):
            os.makedirs(f"gradcam/{lesion_image['guess']}")  
    
    cv2.imwrite(f"gradcam/{lesion_image['guess']}/original.png", cv2.cvtColor(np.array(i), cv2.COLOR_RGB2BGR))

    for model_configs in models:

        model = set_model(model_configs['name'], num_class=3)

        ckpt = torch.load(model_configs['path'],map_location=torch.device('cpu'))
        model.load_state_dict(ckpt['model_state_dict'])

        img_tensor = ImgEvalTransform(size=model.default_cfg['input_size'][1:], normalization=(model.default_cfg['mean'], model.default_cfg['std']),)(i)

        if model_configs["name"] == "resnest101e":
            target_layers = [model.layer4[-1].conv3] # this is the last conv layer of the last block
        elif model_configs["name"] == "efficientnet_b2":
            target_layers = [model.blocks[-1][-1].conv_pwl] # this is the last conv layer of the last block
        elif model_configs["name"] == "tf_efficientnet_b5":
            target_layers = [model.blocks[-1][-1].conv_pwl]

        cam = GradCAM(model=model, target_layers=target_layers)

        targets = [ClassifierOutputTarget(lesion_image['class'])]

        grayscale_cam = cam(input_tensor=img_tensor.unsqueeze(0), targets=targets)

        grayscale_cam = grayscale_cam[0, :]

        grayscale_cam = (grayscale_cam-np.min(grayscale_cam))/(np.max(grayscale_cam)-np.min(grayscale_cam))

        img_tensor = img_tensor.permute(1, 2, 0).numpy().astype(np.float32)

        img_tensor = (img_tensor-np.min(img_tensor))/(np.max(img_tensor)-np.min(img_tensor))

        visualization = show_cam_on_image(img_tensor, grayscale_cam, use_rgb=True)

        cv2.imwrite(f"gradcam/{lesion_image['guess']}/gradcam_{model_configs['name']}.png", cv2.cvtColor(visualization, cv2.COLOR_RGB2BGR))

